In [1]:
import os
import numpy as np
import trimesh
import pandas as pd

In [15]:
def farthest_point_sampling(points, num_samples):
    """Perform farthest point sampling (FPS) to get fixed number of points."""
    selected_pts = [points[np.random.randint(len(points))]]
    for _ in range(num_samples - 1):
        distances = np.linalg.norm(points - selected_pts[-1], axis=1)
        farthest_idx = np.argmax(distances)
        selected_pts.append(points[farthest_idx])
    return np.array(selected_pts)

def preprocess_and_save(obj_filepaths, output_dir, num_points=1024):
    os.makedirs(output_dir, exist_ok=True)

    for file_path in obj_filepaths:
        mesh = trimesh.load_mesh(file_path)
        
        # Try sampling points
        points, _ = trimesh.sample.sample_surface_even(mesh, num_points)
        
        # If not enough points, pad with random duplicates
        if len(points) < num_points:
            indices = np.random.choice(len(points), num_points - len(points), replace=True)
            extra_points = points[indices]
            points = np.vstack([points, extra_points])
        
        # If too many points, use FPS to downsample
        elif len(points) > num_points:
            points = farthest_point_sampling(points, num_points)
        
        # Normalize the point cloud
        centroid = np.mean(points, axis=0)
        points -= centroid
        scale = np.max(np.linalg.norm(points, axis=1))
        points /= scale
        
        # Save the fixed-size point cloud
        filename = os.path.splitext(os.path.basename(file_path))[0] + ".npy"
        np.save(os.path.join(output_dir, filename), points)

    print(f"Processed {len(obj_filepaths)} meshes and saved point clouds to {output_dir}")



In [16]:
df = pd.read_csv("Data/ShapeNetSem/Datasets/subset_template_200.csv")

In [17]:
mesh_ids = df['fullId'].to_list()
mesh_dir = 'Data/ShapeNetSem/Files/models-OBJ/models/'
mesh_paths = [f'{mesh_dir}{id}.obj' for id in mesh_ids if os.path.isfile(f'{mesh_dir}{id}.obj')]

In [18]:
print(len(mesh_paths))

200


In [19]:
output_folder = "Data/ProcessedData/PointClouds"
preprocess_and_save(mesh_paths, output_folder)

only got 477/1024 samples!
only got 1023/1024 samples!
only got 772/1024 samples!
only got 961/1024 samples!
only got 921/1024 samples!
only got 981/1024 samples!
only got 435/1024 samples!
only got 337/1024 samples!
only got 477/1024 samples!
only got 683/1024 samples!
only got 592/1024 samples!
only got 487/1024 samples!
only got 407/1024 samples!
only got 303/1024 samples!
only got 117/1024 samples!
only got 415/1024 samples!
only got 216/1024 samples!
only got 203/1024 samples!
only got 421/1024 samples!
only got 938/1024 samples!
only got 97/1024 samples!
only got 92/1024 samples!
only got 331/1024 samples!
only got 660/1024 samples!
only got 907/1024 samples!
only got 127/1024 samples!
only got 662/1024 samples!
only got 818/1024 samples!
only got 621/1024 samples!
only got 400/1024 samples!
only got 422/1024 samples!
only got 856/1024 samples!
only got 591/1024 samples!
only got 902/1024 samples!
only got 309/1024 samples!
only got 635/1024 samples!
only got 574/1024 samples!
on

Processed 200 meshes and saved point clouds to Data/ProcessedData/PointClouds


In [21]:
import os
import numpy as np
import trimesh

def farthest_point_sampling(points, num_samples):
    """Performs farthest point sampling (FPS) to ensure uniform point distribution."""
    selected_pts = [points[np.random.randint(len(points))]]
    for _ in range(num_samples - 1):
        distances = np.linalg.norm(points - selected_pts[-1], axis=1)
        farthest_idx = np.argmax(distances)
        selected_pts.append(points[farthest_idx])
    return np.array(selected_pts)

def load_binvox_as_point_cloud(binvox_path, num_points=1024):
    """Load a BINVOX file and extract a point cloud using trimesh."""
    try:
        # Load the binvox file as a VoxelGrid
        voxel_grid = trimesh.exchange.binvox.load_binvox(binvox_path)
        points = voxel_grid.points  # Extract voxel positions
        
        if len(points) < num_points:
            indices = np.random.choice(len(points), num_points - len(points), replace=True)
            points = np.vstack([points, points[indices]])

        if len(points) > num_points:
            points = farthest_point_sampling(points, num_points)

        return points
    except Exception as e:
        print(f"Error loading BINVOX file {binvox_path}: {e}")
        return None

def preprocess_and_save(mesh_ids, obj_dir, binvox_dir, output_dir, num_points=1024):
    os.makedirs(output_dir, exist_ok=True)

    for mesh_id in mesh_ids:
        obj_path = os.path.join(obj_dir, mesh_id + ".obj")
        binvox_path = os.path.join(binvox_dir, mesh_id + ".binvox")

        points = None

        if os.path.exists(obj_path):
            mesh = trimesh.load_mesh(obj_path)
            points, _ = trimesh.sample.sample_surface(mesh, num_points * 4)

        if (points is None or len(points) < num_points) and os.path.exists(binvox_path):
            points = load_binvox_as_point_cloud(binvox_path, num_points)

        if points is None or len(points) < num_points:
            continue  # Skip if neither method produces enough points

        if len(points) > num_points:
            points = farthest_point_sampling(points, num_points)

        centroid = np.mean(points, axis=0)
        points -= centroid
        scale = np.max(np.linalg.norm(points, axis=1))
        points /= scale

        filename = mesh_id + ".npy"
        np.save(os.path.join(output_dir, filename), points)

    print(f"Processed {len(mesh_ids)} meshes and saved point clouds to {output_dir}")

In [22]:
obj_directory = "Data/ShapeNetSem/Files/models-OBJ/models"
binvox_directory = "Data/ShapeNetSem/Files/models-binvox"
output_directory = "Data/ProcessedData/PointCloudswithBinvox"
preprocess_and_save(mesh_ids, obj_directory, binvox_directory, output_directory)

Processed 200 meshes and saved point clouds to Data/ProcessedData/PointCloudswithBinvox


In [1]:
from torch.utils.data import Dataset

class PointBertPCModalityDataset(Dataset):
    """Creates text modality dataset for ShapeNetSem with CLIP Tokenizer"""

    def __init__(self, dataset_path, pc_dir):
        super().__init__()
        self.dataframe = pd.read_csv(dataset_path)
        self.pc_dir = pc_dir
        

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        """
        Returns:
            idx (int): Index
            tokenized_text (torch.Tensor): Tokenized text for CLIP
            text_prompt (str): The actual text prompt
        """
        mesh_id = self.dataframe.loc[idx, 'fullId']

        

        return idx, tokenized_text.squeeze(0), text_prompt